## 神经网络的学习

---

#### 介绍

本实验先介绍了使用穷举方法来求解权重的不可行性，并且引出了梯度下降的概念。以简单的求解线性函数的例子出发，以最容易理解的方式介绍了梯度下降的概念以及步骤。然后探讨了，梯度下降中相关参数对最终结果的影响。最终将梯度下降算法带入神经网络中，详细的阐述了，利用该算法更新网络权重的具体步骤。

#### 知识点

- 暴力破解权重的不可行性
- 梯度下降法
- 梯度下降的步长
- 梯度的计算
- 梯度下降在神经网络中的应用

---

### 暴力法求解权重

上一章节，我们一直在讨论每个节点的损失误差的问题，还没有直接谈到权重的更新方法。在解开这个秘密之前，我们还需要理解一个重要的思想

到目前为止，我们已经懂得了，如何让误差反向传播到网络中的每一层。为什么需要学习这个呢？因为我们需要使用误差来指导链接权重的调整，从而改进神经网络输出的总体答案。这其实上就是第一章的线性分类器所做的事情。

但是，这些节点都不是简单的线性分类器。这些复杂的节点对加权后的信号进行求和，并应用 S 阈值函数，将所得到的结果输出给下一层的节点。因此，我们如何才能有效地更新这些复杂节点的链接权重呢？这里，我们尝试使用微妙的代数式来对权重进行求解。

让我们观察一下，下面这个“面目可憎”的表达式，这是一个简单的 3 层、每层 3 个节点的神经网络，其中输入层节点的输出是一个关于输入值和链接权重的函数。在节点 $i$ 处的输入是 $x_i$ ，连接输入层节点 $i$ 到隐藏层节点 $j$ 的链接权重为 $w_{i,j}$ ，类似地，隐藏层节点 $j$ 的输出是 $x_j$ ，连接隐藏层节点 $j$ 和输出层节点 $k$ 的链接权重是 $w_{j ,k}$  。那个看似有趣的符号 $Σ^b_a$ 意味着对在 $a$ 和 $b$ 值之间的所有后续表达式求和。 

<img width="500px" src="https://doc.shiyanlou.com/courses/uid1166617-20191216-1576476235123/wm">

哎呀！这些公式让人头皮发麻，因此我们还是不要硬碰硬地去求解这个表达式了。

首先，我们可以采用随机组合权重的方法，去寻找最佳的权重组合。也就是说将所有的可能性都放入等式中，试一遍，总能找一个成立的。我们称这种方法为暴力方法。

假设现在的每个权重在 $-1$ 和 $+1$ 之间有 $1000$ 种可能的值，如 $0.501$、$-0.203$ 和 $0.999$ 。那么对于 $3$ 层、每层 $3$ 个节点的神经网络，我们可以得到 $18$ 个权重，因此共有 $18000$ 种可能性需要测试。一般一个相对典型的神经网络，每层有 $500$ 个节点，那么我们需要测试 $5$ 亿种权重的可能性。如果每组组合需要花费 $1$ 秒钟进行计算，那么对于一个训练样本，我们需要花费 $16$ 年来更新权重！对于 $1000$ 种训练样本，我们要花费 $16000$ 年！ 

因此暴力方法求解权重不切实际。

数学家多年来都未解决这个难题，直到 $20$ 世纪 $60$ 年代到 $70$ 年代，这个难题才有了切实可行的求解办法。虽然对于谁最先解出了这个难题或做出了关键性的突破，大家莫衷一是。但，重要的一点是，这个迟来的发现导致了现代神经网络爆炸性的发展，并使现在的神经网络可以执行一些令人印象深刻的任务。

那么，我们如何解决这样一个明显的难题呢？这是解决方法在之前，我们已经用到了，接下来我们对它进行系统的阐述。

### 梯度下降法求解权重

#### 梯度下降法的概念

让我们来详细的解释一下这是什么意思。想象一下，你身处在一个地形复杂，有波峰波谷，在黑暗中，伸手不见五指的地方。你仅仅知道的是你在一个山坡上，你需要到达坡底。


对于整个地形，你没有精确的地图，只有一把手电筒。你能做什么呢？你可能会使用手电筒，做近距离的观察，但你肯定看不到整个地形。你所能看到是你周围的土地中，那个方向是下坡。于是你就小步地往这个方向走。这种方式下，你不需要完整的地图，也不需要事先制定路线，你只需一步一个脚印，缓慢地前进，就一定能够慢慢地下山。

<img width="500px" src="https://doc.shiyanlou.com/courses/uid1166617-20191216-1576479350467/wm">

在数学上，这种方法称为梯度下降（Gradient Descent）。在你迈出一步之后，再次观察周围的地形，看看你下一步往哪个方向走，才能更接近目标。然后，你就往那个方向走一步。你一直保持这种方式，就能够到达山底。梯度是指地面的坡度。你走的方向应该是最陡的，且坡度向下的方向。

现在，想象一下，这个复杂的地形是一个数学函数。梯度下降法给我们带来一种能力，让我们无需从数学上对复杂函数进行求解，就可以找到最小值。当然，由于我们采用步进的方式接近答案，一点一点地改进所在的位置，因此这可能无法给出精确解。总之，我们可以使用更小的步子来朝着实际的最小值方向迈进，优化答案，直到我们对于所得到的精度感到满意为止。

这种酷炫的梯度下降法与神经网络之间有什么联系呢？好吧，如果我们将复杂困难的函数当作网络误差，那么下山找到最小值就意味着最小化误差。这样我们就可以改进网络输出。这就是我们希望做到的！

#### 梯度下降算法的步长

为了正确理解梯度下降的思想，我们需要使用一个超级简单的例子来进行演示。

下图显示了一个简单的函数 $y =(x-1)^2 + 1$ 。如果在这个函数中， $y$ 表示误差，我们希望的是找到合适的 $x$ 的值，使 $y$ 最小。现在，我们假装这不是一个简单的函数，而是一个复杂困难的函数。

<img width="300px" src="https://doc.shiyanlou.com/courses/uid1166617-20191216-1576480697768/wm">

要应用梯度下降的方法，我们必须找一个起点。上图随机选择了一个点作为起点（红色的点）。就像登山者，我们正站在这个地方，环顾四周，观察哪个方向是向下的。在图上标记了当前情况下的斜率，其斜率为负。我们希望沿着向下的方向，因此我们沿着 $x$ 轴向右。也就是说，我们稍微地增加 $x$ 的值。这是登山者的第一步。你可以观察到，我们改进了我们的位置，向实际最小值靠近了一些。

我们假设从某个地方开始，如下图所示。

<img width="300px" src="https://doc.shiyanlou.com/courses/uid1166617-20191216-1576480832161/wm">

这一次，我们所在之处的斜率为正，因此我们向左移动。也就是说，我们稍微减小 $x$ 值。同样，你可以观察到我们已经改善了位置，向真实的最小值靠近了一些。我们可以继续这样操作，直到几乎不能改进为止，这样我们就确信已经到达了最小值。

我们要改变步子大小，来避免超调，这样就可以避免在最小值的地方来回反弹了。为了更快到达最小值，我们可以在开始的时候将步子迈大一点，但是当我们身处最小值点附近的时候，我们就需要将步子迈小一点。也就是说让步子的大小，和梯度的大小成正比。如下图所示，当步子跟着梯度进行变化时，我们就能够更好的到达最小值。

<img width="300px" src="https://doc.shiyanlou.com/courses/uid1166617-20191216-1576481381539/wm">

当函数有很多参数时，这种方法才真正地显现出它的亮点。比如说，神经网络中的误差函数取决于许多的权重参数，这些参数通常有数百个。对于这么多的参数，用梯度下降算法，显然是一个比较好的选择。

#### 解决局部最小问题

同样，下面我们将使用稍微复杂的、依赖 2 个参数的函数，详细说明梯度下降法。这其实就是一个三维空间，其中高为函数的 y 值。长和宽为函数的两个参数变量。

<img width="300px" src="https://doc.shiyanlou.com/courses/uid1166617-20191216-1576482609901/wm">

观察这个三维曲面，你可以再次思考，梯度下降是否会终止于右侧的另一个山谷。话句话说，由于复杂的函数有众多的山谷，梯度下降有时会卡在错误的山谷中吗？答案是肯定的，这种情况可能会发生，也就是我们所到达的山谷可能不是最低的山谷。

为了避免终止于错误的山谷或错误的函数最小值，我们从山上的不同点开始，多次训练神经网络，确保并不总是终止于错误的山谷。不同的起始点意味着选择不同的起始参数，在神经网络的情况下，这意味着选择不同的起始权重。 

下面详细说明了使用梯度下降方法的三种不同尝试，其中有一次，这种方法终止于错误的山谷中。当我在确定最佳的权重是，只需要把这几次的尝试进行比较，找到最小的 y 值所对应的权重即可。

<img width="500px" src="https://doc.shiyanlou.com/courses/uid1166617-20191216-1576483932638/wm">

让我们暂停，总结一下梯度下降算法的好处。


- 梯度下降法是求解函数最小值的一种很好的办法，当函数非常复杂困难，并且不能轻易使用数学代数求解函数时，这种方法可以发挥很好的作用。
- 当函数有很多参数，一些其他方法不切实际，或者会得出错误答案，这种方法依然可以适用。
- 这种方法也具有弹性，可以容忍不完善的数据，如果我们不能完美地描述函数，或我们偶尔意外地走错了一步，也不会错得离谱。


#### 梯度下降算法的应用

神经网络的输出是一个极其复杂困难的函数，这个函数具有许多参数。我们可以使用梯度下降法，计算出正确的权重吗？只要我们选择了合适的误差函数，就可以找到最佳权重。

神经网络本身的输出函数不是一个误差函数。但我们知道，由于误差是目标训练值与实际输出值之间的差值，因此我们可以很容易地把输出函数变成误差函数。

此处，我们要注意一些事情。观察下表，这是 3 个输出节点的目标值和实际值以及误差函数的候选项。

| 网络输出 	| 目标输出 	| 误差 = 目标值-实际值  	| 误差 = $|$目标值-实际值$|$ 	| 误差 = ( 目标值-实际值 $)^2$ 	|
|:--------:	|:--------:	|:----------------------:	|:------------------------:	|:----------------------:	|
|    0.4   	|    0.5   	|           0.1          	|            0.1           	|          0.01          	|
|    0.8   	|    0.7   	|          -0.1          	|            0.1           	|          0.01          	|
|    1.0   	|    1.0   	|            0           	|             0            	|            0           	|
|     求和   	||            0           	|            0.2           	|          0.02          	|

误差函数的第一个候选项是（ 目标值 - 实际值 ），这似乎足够合理了，对吧？但是，如果你观察对所有节点的误差之和，由于误差的正负抵消，误差和为 0。总和为零意味着没有误差，可是这明显是存在误差的呀。因此，我们发现这不是一个很好的测量方法。

为了纠正这一点，我们采用差的绝对值，即将其写成 | 目标值 - 实际值 | 来作为误差函数 。这样我们就无需担心误差正负抵消的问题了。但是，由于斜率在最小值点附近不是连续的（ 即导数不存在 ），这使得梯度下降方法无法很好地发挥作用，因此这个误差函数没有得到广泛应用。并且，在这种情况下，斜率的是不会发生变化的，那么就导致步长不变，就可能出现超调的现象。

第三种选择是差的平方，即 ( 目标值-实际值 $)^2$ 。我们更喜欢使用第三种误差函数，而不喜欢使用第二种误差函数，原因有以下几点：

- 使用误差的平方，我们可以很容易使用代数计算出梯度下降的斜率。 
- 误差函数平滑连续，这使得梯度下降法很好地发挥作用——没有间断点，也没有跳跃点。 
- 越接近最小值，梯度越小，这意味着，如果我们使用这个函数调节步长，超调的风险就会变得较小。

那么，是否有第四个选项呢？有，你可以构造各种各样的复杂有趣的代价函数。一些函数可能完全行不通，一些函数可能对特定类型的问题能起到显著的作用。因此并没有任何一个误差函数可以解决所有的问题，具体问题应当具体分析。

现在，我们已经跑到了最后一圈，即将到达终点了！ 

要使用梯度下降的方法，现在我们需要计算出误差函数相对于权重的斜率，也就是上文我们提到的梯度。这需要微积分的知。微积分用数学方式精确的阐述了，当一些变量改变时，其他应变量会如何改变。例如，当在弹簧上施加一个伸展力时，弹簧的长度如何变化。此处，我们感兴趣的是，误差函数是如何依赖于神经网络中的链接权重的。

既然我们明白了想要的目标，而画图总是有助于让我们脚踏实地，那么我们就从图形开始吧。

<img width="300px" src="https://doc.shiyanlou.com/courses/uid1166617-20191216-1576489037076/wm">

此图与我们先前看到的图一样，主要是强调我们所做的事情与之前并没有什么不同，只是将目标函数换成了神经网络的误差函数。我们试图优化的参数是网络的链接权重。在这个简单的例子中，我们只演示了一个权重，但是我们知道神经网络有众多权重参数。

下图显示了两个链接权重，这次，误差函数是一个三维曲面。这个曲面随着两个链接权重的变化而变化。其实，我们努力最小化误差就像是在多山的地形中寻找一个山谷。

<img width="300px" src="https://doc.shiyanlou.com/courses/uid1166617-20191216-1576489365833/wm">

当函数具有多个参数时（多维空间），要画出误差曲面相对较难，但是使用梯度下降寻找最小值的思想是相同的。

让我们使用数学的方式，写下想要取得的目标。

<img width="60px" src="https://doc.shiyanlou.com/courses/uid1166617-20191216-1576489793476/wm">

这个表达式表示了当权重 $w_{j,k}$ 改变时，误差 $E$ 是如何改变的。这是误差函数的斜率，也就是我们希望使用梯度下降的方法到达最小值的方向。

在我们求解表达式之前，我们将重回输入层和隐藏层之间的链接权重，如下图所示：

<img width="400px" src="https://doc.shiyanlou.com/courses/uid1166617-20191216-1576490151190/wm">

在进行微积分计算时，我们会时不时地返回来参照此图，以确保我们没有忘记每个符号的真正含义。读者请不要被吓倒而裹足不前，这个过程并不困难，我们还会注意对它们进行解释。

#### 梯度的计算

首先，让我们展开误差函数，这里我们将所有数据的目标值与实际值之差的平方和作为总误差函数。下图为误差函数 $E$ 关于参数 $w_{j,k}$ 的偏导（也就是上面所说的梯度）的函数式。

<img width="300px" src="https://doc.shiyanlou.com/courses/uid1166617-20191217-1576544632266/wm">

注意，节点 $n$ 的输出 $o_n$ 只取决于连接到这个节点的链接。因此节点 $k$ 的输出 $o_k$ 只取决于权重 $w_{j,k}$，而与与 $w_{j,a}，w_{j,b}...$等链接在其他输出节点上的权重无关。

从数学角度来说，就是 函数 $E$ 中有些项与 $w_{j,k}$ 没有关系，那么这个时候他们关于 $w_{j,k}$ 求出的偏导就为 0 。

如果你喝了咖啡，头脑比较清醒，你可能已经意识到，这意味着误差函数根本就不需要对所有输出节点求和。原因是节点的输出只取决于自己所链接的权重。因此，在求取关于 $w_{j,k}$ 的偏导时，我们就可以忽略除了 $w_{j,k}$ 所连接的节点之外的其它节点的输出。

无论如何，我们现在有了一个相对简单的表达式了。

<img width="300px" src="https://doc.shiyanlou.com/courses/uid1166617-20191217-1576548018083/wm">

现在，我们将进行一点微积分计算。

 $t_k$（数据的实际值）的部分是一个常数，因此它不会随着 $w_{j，k}$ 的变化而变化。也就是说， $t_k$ 不是 $w_{j，k}$ 的函数。由于我们使用权重前馈信号，得到输出值 $o_k$。因此 $o_k$ 肯定依赖与 $w_{j,k} $。

我们将使用链式法则，将这个微积分任务分解成更多易于管理的小块。

<img width="300px" src="https://doc.shiyanlou.com/courses/uid1166617-20191217-1576548867112/wm">

现在，我们可以反过来对相对简单的部分各个击破。我们对平方函数进行简单的微分，就很容易击破了第一个简单的项。这使我们得到了以下的式子

<img width="300px" src="https://doc.shiyanlou.com/courses/uid1166617-20191217-1576548949863/wm">

对于第二项，我们需要仔细考虑一下，但是无需考虑过久。 $o_k$ 是节点 $k$ 的输出。如果你还记得，这是在连接输入信号上进行加权求和，然后对所得结果使用 $S$ 函数进行处理，最后得到输出 $o_k$ 。让我们将这写下来（将 $o_k$ 的值带换成上述过程），清楚地表达出来。

<img width="500px" src="https://doc.shiyanlou.com/courses/uid1166617-20191217-1576549019027/wm">

$o_j$ 是前一个隐藏层节点的输出， $o_k$ 为最终层的输出。

我们如何微分 $S$ 函数呢？ 对 $S$ 函数求微分，这对我们而言是一种非常艰辛的方法，但是，其他人已经完成了这项工作。我们可以只使用众所周知的答案，就像全世界的数学家每天都在做的事情一样。

<img width="500px" src="https://doc.shiyanlou.com/courses/uid1166617-20191217-1576549430799/wm">

在微分后，一些函数变成了非常可怕的表达式。$S$ 函数微分后，可以得到一个非常简单、易于使用的结果。在神经网络中，这是 $S$ 函数成为大受欢迎的激活函数的一个重要原因。

因此，让我们再次使用链式法则，化简上面的函数，得到以下的表达式。

<img width="600px" src="https://doc.shiyanlou.com/courses/uid1166617-20191217-1576549786408/wm">

这个额外的最后一项是什么呢？由于在 $ Sigmod()$ 函数内部的表达式也需要对 $w_{j,k}$ 进行微分，因此我们对 $S$ 函数微分项再次应用链式法则。这也非常容易，答案很简单，为 $o_j$ 。

在写下最后的答案之前，让我们把在前面的 2 去掉。由于这个 2 是所有的值都会乘的，因此大的斜率依然会大，小的斜率依然会小。只要我们牢牢记住需要什么，在表达式前面的常数，无论是 $2$、$3$ 还是 $100$，都无关紧要。因此，让我们去掉这个常数，使事情变得更简单些。

这就是我们一直在努力要得到的最后答案，这个表达式描述了误差函数的斜率，也就是误差函数的梯度变化情况，这样我们就可以调整权重 $w_{j，k}$ 了。

<img width="500px" src="https://doc.shiyanlou.com/courses/uid1166617-20191217-1576550550623/wm">

嘿！我们成功做到了！

这就是我们一直在寻找的神奇表达式，也是训练神经网络的关键。

这个表达式值得再次回味，颜色标记有助于显示出表达式的各个部分：

- 第一部分，（ 目标值 - 实际值 ），就是误差函数对输出信号 $o_k$ 求偏导得到的。
- 第二部分，就是对 $Sigmod$ 函数求偏导得到的。
- 第三部分，就是对 $Sigmod$ 函数中的求和表达式求偏导得到的。

我们还需要做最后一件事情。我们所得到的这个表达式，是为了优化隐藏层和输出层之间的权重。现在，我们需要完成工作是，为输入层和隐藏层之间的权重找到类似的误差斜率。

同样，我们可以进行大量的代数运算，但是不必这样做。我们可以很简单地使用刚才所做的解释，为感兴趣的新权重集重新构建一个表达式。

- 第一部分的（ 目标值 - 实际值 ）误差，现在变成了隐藏层节点中重组的向后传播误差（上一章节已经系统阐述了它的求解过程）。正如在前面所看到的那样。我们称之为 $e_j$。 
- $Sigmoid$ 部分可以保持不变，但是内部的求和表达式指的是前一层，因此求和的范围是所有由权重调节的进入隐藏层节点 $j$ 的输入。我们可以称之为 $i_j$。 
- 现在，最后一部分是第一层节点的输出 $o_i$，这碰巧是输入信号。 

我们简单的利用问题中的对称性，构建了一个新的表达式，避免了大量的工作。这种方法虽然很简单，但却是一种很强大的技术，一些天赋异禀的数学家和科学家都使用这种技术。你肯定可以使用这个技术，给你的队友留下深刻印象。

因此，我们一直在努力达成的最终答案的第二部分，如下所示：

<img width="500px" src="https://doc.shiyanlou.com/courses/uid1166617-20191217-1576552611111/wm">

这是我们所得到误差函数斜率，用于输入层和隐藏层之间权重调整。现在，我们得到了关于斜率的所有关键表达式。我们可以使用这些表达式，在应用每层训练样本后，对权重进行更新了。

#### 权重的更新

权重改变的方向与梯度方向相反，因此下图中，权重的更新方向为梯度的相反数。同时也为了保证权重不会由于持续的超调而在最小值附近来回摆动，我们就引入了学习因子（其实就是我们之前讲的步长）的概念。

下图为利用梯度下降法，更新参数的公式：

<img width="500px" src="https://doc.shiyanlou.com/courses/uid1166617-20191217-1576552978979/wm">

更新后的权重 $w_{j,k}$ 是通过对误差斜率的取反，来调整旧的权重而得到的。正如我们先前所看到的，如果斜率为正，我们希望减小权重。如果斜率为负，我们希望增加权重。因此，我们需要对斜率取反。

符号 $α$ 是一个因子，这个因子可以调节变化的强度，确保不会超调。我们通常称这个因子为学习率,又叫步长。

这个表达式不仅适用于隐藏层和输出层之间的权重，更适用于输入层和隐藏层之间的权重。

如果我们试图按照矩阵乘法的形式进行运算，那么我们需要看看计算的过程。为了有助于理解，我们将按照以前那样，写出权重变化矩阵的每个元素。

<img width="500px" src="https://doc.shiyanlou.com/courses/uid1166617-20191217-1576560649006/wm">

由于学习率只是一个常数，并没有改变矩阵乘法的总体结构，因此这里我们先省略学习率 $α$。

权重改变矩阵中包含的值，这些值可以调整链接权重 $w_{j，k}$ ，这个权重链接了当前层节点 $j$ 与下一层节点 $k$。你可以发现，表达式中的第一项使用下一层（节点 $k$ ）的值，最后一项使用前一层（节点 $j$ ）的值。

仔细观察上图，你就会发现，表达式的最后一部分，也就是单行的水平矩阵，是前一层 $o_j$ 的输出的转置。

因此，权重更新矩阵有如下的矩阵形式，这种形式可以让我们通过计算机编程语言高效地实现矩阵运算。

<img width="500px" src="https://doc.shiyanlou.com/courses/uid1166617-20191217-1576561020596/wm">

实际上，这不是什么复杂的表达式。 $Sigmoid$ 的输出其实就是 $O_k$。

好了！任务完成。这就是权重的更新的全过程。简单的说，每层权重的变化量都是一个 $\triangle w_{j,k}$ ，而这个变化量只需要通过上面的公式进行计算即可。

### 实验总结

- 神经网络的误差是内部链接权重的函数。
- 改进神经网络，意味着通过改变权重减少这种误差。
- 梯度下降算法其实就是，通过误差函数的梯度下降，采取小步长，迭代地改进权重的过程。
- 使用微积分可以很容易地计算出误差斜率。

<hr><div style="color: #999; font-size: 12px;"><i class="fa fa-copyright" aria-hidden="true"> 本课程内容版权归实验楼所有，禁止转载、下载及非法传播。</i></div>